In [ ]:
import requests
import json
import pandas as pd
import re

def isValidJson(myjson):
  try:
    json_object = json.loads(myjson)
  except ValueError as e:
    return False
  return True

def getRequest(query):
    try:
        response = requests.get('https://search.unbxd.io/fb853e3332f2645fac9d71dc63e09ec1/demo-unbxd700181503576558/search', params=query)
        response.raise_for_status()
        # Code here will only run if the request is successful
        return response.json()
    except requests.exceptions.HTTPError as errh:
        print(errh)
        return None
    except requests.exceptions.ConnectionError as errc:
        print(errc)
        return None
    except requests.exceptions.Timeout as errt:
        print(errt)
        return None
    except requests.exceptions.RequestException as err:
        print(err)
        return None

def parseValue(val):
    tmp = ""
    tmp = str(val)
    tmp = tmp.replace("\"", "\'")
    redata = re.compile(re.escape('true'), re.IGNORECASE)
    tmp = redata.sub('YES', tmp)
    redata = re.compile(re.escape('false'), re.IGNORECASE)
    tmp = redata.sub('NO', tmp)
    return tmp

def productJSONToCSV(product_json, columns):
    result = ""
    for ele in columns:
        val = sorted_json[ele]
        result += "\""
        if isinstance(val, list):
            for ele1 in list(set(val)):
                if ele == "unbxd_color_for_category":
                    result += ele1.split("::")[0] + ", "
                else:
                    result += parseValue(ele1) + ", "
            result = result[:-2]
        else:
            result += parseValue(val)
        result += "\","
    result = result[:-1]
    return result

query = {'q':'*', 'rows':'1', 'start':'0'}
resp = getRequest(query)
n = resp["response"]["numberOfProducts"]
# print(n)

results = []
c = 0

result = ""

sorted_json = json.loads(json.dumps(resp["response"]["products"][0], sort_keys=True))

for ele in sorted_json:
    result += str(ele)
    result += ","
result = result[:-1] + "\n"

while c <= n:
    query = {"q":"*", "rows":"100", "start":c}
    resp = getRequest(query)

    if resp == None:
        print('Error: Empty response at start = ' + str(c))
        break

    for arr in resp["response"]["products"]:
        if results.count(arr) == 0:
            result += productJSONToCSV(arr, sorted_json)
            result += "\n"

    # print('Appended: ' + str(i))
    c = c + 100

# print(result)

with open("Output.csv", "w") as text_file:
    text_file.write(result)


